In [300]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
sift = cv2.SIFT_create(nfeatures=25)

In [301]:
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
sift_image = cv2.drawKeypoints(gray, keypoints, img)
cv2.imwrite("table-sift.jpg", sift_image)

True

In [302]:
def get_clusters(n_clusters, img_list):
    kmeans = KMeans(n_clusters=n_clusters).fit(img_list)
    return kmeans

In [303]:
def get_sift_descriptors(sift, img):
    keypoints, detectors = sift.detectAndCompute(img, None)
    return detectors

In [304]:
import os
types = os.listdir("the2_data/train")
vocab = []
for t in types:
    fnames = next(os.walk(f'the2_data/train/{t}'), (None, None, []))[2]
    for file in fnames:
        img = cv2.imread(f'the2_data/train/{t}/{file}')
        detectors = get_sift_descriptors(sift, img)
        if detectors is not None:
            for d in detectors:
                vocab.append(d)

In [305]:
num_of_classes = len(types)
kmeans = get_clusters(128, vocab)

In [306]:
bow = {}
for t in types:
    fnames = next(os.walk(f'the2_data/train/{t}'), (None, None, []))[2]
    hists = []
    for file in fnames:
        img = cv2.imread(f'the2_data/train/{t}/{file}')
        detectors = get_sift_descriptors(sift, img)
        if detectors is not None:
            det_list = detectors.astype(float)
            predictions = kmeans.predict(det_list)
            hist = np.histogram(predictions, bins=list(range(128)))[0]
            hists.append(hist/np.linalg.norm(hist))
    bow[t] = hists

In [307]:
def predict_by_knn(k, bow, arr):
    types = bow.keys()
    nns = [99999] * k
    labels = [''] * k
    for t in types:
        for point in bow[t]:
            dist = get_point_distance(arr, point)
            max_n_index = nns.index(max(nns))
            if dist < nns[max_n_index]:
                nns[max_n_index] = dist
                labels[max_n_index] = t

    return nns, labels

In [308]:
img1 = cv2.imread('the2_data/validation/road/0001.png')
img2 = cv2.imread('the2_data/validation/road/0001.png')
keypoints1, detectors1 = sift.detectAndCompute(img1, None)
keypoints2, detectors2 = sift.detectAndCompute(img2, None)

In [309]:
predictions1 = kmeans.predict(detectors1.astype(float))
predictions2 = kmeans.predict(detectors2.astype(float))
print(predictions1)

[123  77  77  77   4]


In [310]:
hh1 = np.histogram(predictions1, bins=list(range(128)))[0]
hh2 = np.histogram(predictions2, bins=list(range(128)))[0]

In [311]:
a = predict_by_knn(8, bow, hh1)[1]
print(a)
max(set(a), key = a.count)

['cup', 'crab', 'road', 'aquarium_fish', 'road', 'beetle', 'road', 'beetle']


'road'

In [329]:
def get_acc(bow, kmeans, kk, sift, types, knn):
    count = 0
    correct = 0
    pred_list = []
    truth_list = []
    for t in types:
        fnames = next(os.walk(f'the2_data/validation/{t}'), (None, None, []))[2]
        for file in fnames:
            img = cv2.imread(f'the2_data/validation/{t}/{file}')
            detectors = get_sift_descriptors(sift, img)
            if detectors is not None:
                pred = kmeans.predict(detectors.astype(float))
                h_hist = np.histogram(pred, bins=list(range(kk)))[0]
                a = predict_by_knn(knn, bow, h_hist)[1]
                result = max(set(a), key = a.count)
                count += 1
                if result == t:
                    correct += 1
                pred_list.append(result)
                truth_list.append(t)
    print(correct, count)
    return (correct/count)*100, pred_list, truth_list

In [317]:
# kmeans 128 nfeatures=25 sift --> 307 1498

In [ ]:
print(get_acc(bow, kmeans, 128, cv2.SIFT_create(nOctaveLayers=12), types, 16))


In [318]:
print(get_acc(bow, kmeans, 128, cv2.SIFT_create(nOctaveLayers=5), types, 8))
print("--------")
print(get_acc(bow, kmeans, 128, cv2.SIFT_create(contrastThreshold=0.08), types, 8))
print("--------")
print(get_acc(bow, kmeans, 128, cv2.SIFT_create(edgeThreshold=20), types, 8))
print("--------")
print(get_acc(bow, kmeans, 128, cv2.SIFT_create(sigma=10), types, 8))
print("--------")


301 1499
20.08005336891261
--------
270 1456
18.543956043956044
--------
306 1498
20.427236315086784
--------
28 229
12.22707423580786
--------


In [330]:
acc16, pred16, truth16 = get_acc(get_bow(types, sift, kmeans, 128), kmeans, 128, cv2.SIFT_create(nOctaveLayers=12), types, 16)
acc32, pred32, truth32 = get_acc(get_bow(types, sift, kmeans, 128), kmeans, 128, cv2.SIFT_create(nOctaveLayers=12), types, 32)
acc64, pred64, truth64 = get_acc(get_bow(types, sift, kmeans, 128), kmeans, 128, cv2.SIFT_create(nOctaveLayers=12), types, 64)

328 1498
348 1498
346 1498


In [333]:
from sklearn.metrics import confusion_matrix
confusion_matrix(truth64, pred64, labels=types)

array([[49,  3,  3,  9,  0,  0,  4, 13, 10,  0,  2,  0,  1,  5,  1],
       [10,  6,  1,  7,  9,  1,  5, 10, 13,  6,  4,  2,  4, 15,  7],
       [ 9,  0, 14, 10,  1,  0,  4, 10,  5,  4, 12,  1,  5, 22,  2],
       [ 8,  5,  2, 20, 12,  4,  7, 18,  3,  3,  1,  0,  2,  8,  7],
       [ 5,  3,  0,  7, 18,  4,  5, 13, 11,  1, 11,  0,  6, 14,  2],
       [10,  4,  3, 12,  3,  1,  4, 11, 21,  6,  6,  3,  3, 12,  1],
       [15,  1,  2, 10,  6,  1, 13, 18, 10,  4,  3,  2,  3, 12,  0],
       [26,  0,  2,  7,  0,  2,  0, 50,  5,  3,  0,  0,  0,  3,  2],
       [ 7,  1,  2,  5,  1,  0,  4,  8, 57,  5,  3,  0,  2,  5,  0],
       [ 8,  0,  1,  4,  2,  2,  1, 13, 35, 15,  6,  2,  1,  6,  3],
       [15,  3,  5,  3,  2,  3,  3, 10, 14,  4, 23,  3,  2,  9,  1],
       [13,  3,  2,  9,  1,  0,  1, 11,  8,  2, 12, 15,  3, 18,  2],
       [18,  2,  2,  5,  0,  2,  3, 17, 12,  4,  9,  3,  4, 18,  1],
       [ 1,  3,  3,  3,  3,  0,  4,  4,  5,  2, 12,  0,  0, 58,  2],
       [11,  8,  1, 12,  6,  1,  7

In [324]:
def get_bow(types, sift, kmeans, kk):
    bow = {}
    for t in types:
        fnames = next(os.walk(f'the2_data/train/{t}'), (None, None, []))[2]
        hists = []
        for file in fnames:
            img = cv2.imread(f'the2_data/train/{t}/{file}')
            detectors = get_sift_descriptors(sift, img)
            if detectors is not None:
                det_list = detectors.astype(float)
                predictions = kmeans.predict(det_list)
                hist = np.histogram(predictions, bins=list(range(kk)))[0]
                hists.append(hist/np.linalg.norm(hist))
        bow[t] = hists
    return bow

In [327]:
print(get_acc(get_bow(types, sift, kmeans, 256), kmeans, 256, cv2.SIFT_create(nfeatures=25), types, 8))
print(get_acc(get_bow(types, sift, kmeans, 512), kmeans, 512, cv2.SIFT_create(nfeatures=25), types, 8))
#20.56 cıktı noctave 12 icin

307 1498
20.493991989319092
307 1498
20.493991989319092


In [314]:
np.save("cluster_centers",ccenters)

In [45]:
cc = np.load("cluster_centers.npy")

In [71]:
def get_point_distance(arr1, arr2):
    return np.linalg.norm(arr1-arr2)